# SD WebUI All In One Colab
#### Created by [licyk](https://github.com/licyk)
这是一个支持部署多种 WebUI 的 Jupyter Notebook，支持部署以下 WebUI（移除了 Colab 会警告的 WebUI）：
- 1、[InvokeAI](https://github.com/invoke-ai/InvokeAI)
- 2、[Fooocus](https://github.com/lllyasviel/Fooocus)
- 3、[lora-scripts](https://github.com/Akegarasu/lora-scripts)
- 4、[kohya_ss](https://github.com/bmaltais/kohya_ss)

使用时请按顺序执行 Jupyter Notebook 单元。

## 主要功能
1. 功能初始化：导入 SD WebUI All In One 所使用的功能
2. 参数配置：配置安装参数和远程访问方式
3. 应用参数配置：应用已设置的参数
4. 安装：根据配置安装对应的 WebUI
5. 启动：根据配置启动对应的 WebUI

## 其他功能
1. 自定义模型 / 扩展下载配置：设置要下载的模型 / 扩展参数
2. 自定义模型 / 扩展下载：根据配置进行下载模型 / 扩展
3. 更新：将已安装的 WebUI 进行更新

## 提示
1. 在参数配置界面，请填写工作区路径，选择要使用的 WebUI，根据自己的需求选择内网穿透方式（用于访问 WebUI 界面），再根据自己的需求选择模型和扩展。
2. 已知 CloudFlare、Gradio 内网穿透会导致 [Kaggle](https://www.kaggle.com) 平台强制关机，在使用 [Kaggle](https://www.kaggle.com) 平台时请勿勾选这两个选项。
3. 若使用 [Colab](https://colab.research.google.com) 平台，请注意该 Jupyter Notebook 无法在免费版的 Colab 账号中使用，运行前将会收到 Colab 的警告提示，强行运行将会导致 Colab 强制关机（如果 Colab 账号已付费订阅可直接使用该 Jupyter Notebook），可选择仓库中其他的  Jupyter Notebook（将 Colab 中禁止的 WebUI 移除了）。
4. [Ngrok](https://ngrok.com) 内网穿透的稳定性高，使用前需要填写 Ngrok Token，可在 [Ngrok](https://ngrok.com) 官网获取。
5. 在启动时将启动内网穿透，可在控制台输出中看到内网穿透地址，用于访问 WebUI 界面。

In [17]:
#@title 👇 功能初始化
INIT_CONFIG = 1

# 消息格式输出
def echo(msg):
    print(f":: {msg}")



# ARIA2
class ARIA2:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载器
    def aria2(self, url, path, filename):
        import os
        if not os.path.exists(path + "/" + filename):
            echo(f"开始下载 {filename} ，路径: {path}/{filename}")
            !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
            if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"{filename} 下载完成")
            else:
                echo(f"{filename} 下载中断")
        else:
            if os.path.exists(path + "/" + filename + ".aria2"):
                echo(f"开始下载 {filename} ，路径: {path}/{filename}")
                !aria2c --console-log-level=error -c -x 16 -s 16 "{url}" -d "{path}" -o "{filename}"
                if os.path.exists(path + "/" + filename) and not os.path.exists(path + "/" + filename + ".aria2"):
                    echo(f"{filename} 下载完成")
                else:
                    echo(f"{filename} 下载中断")
            else:
                echo(f"{filename} 文件已存在，路径: {path}/{filename}")


    # 大模型下载
    def get_sd_model(self, url, filename):
        pass


    # lora下载
    def get_lora_model(self, url, filename):
        pass


    # 放大模型下载
    def get_upscale_model(self, url, filename):
        pass


    # embedding模型下载
    def get_embedding_model(self, url, filename):
        pass


    # controlnet模型下载
    def get_controlnet_model(self, url, filename):
        pass


    # vae模型下载
    def get_vae_model(self, url, filename):
        pass


    # vae-approx模型下载
    def get_vae_approx_model(self, url, filename):
        pass


    # 获取modelscope下载链接
    def get_modelscope_link(self, origin):
        user = origin.split("/")[0]
        repo = origin.split("/")[1]
        branch = origin.split("/")[2]
        tmp = f"{user}/{repo}/{branch}/"
        file = origin.split(tmp).pop().replace("/", "%2F")
        link = f"https://modelscope.cn/api/v1/models/{user}/{repo}/repo?Revision={branch}&FilePath={file}"
        return link



# GIT
class GIT:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 检测要克隆的项目是否存在于指定路径
    def exists(self, addr=None, path=None, name=None):
        import os
        if addr is not None:
            if path is None and name is None:
                path = os.getcwd() + "/" + addr.split("/").pop().split(".git", 1)[0]
            elif path is None and name is not None:
                path = os.getcwd() + "/" + name
            elif path is not None and name is None:
                path = os.path.normpath(path) + "/" + addr.split("/").pop().split(".git", 1)[0]

        if os.path.exists(path):
            return True
        else:
            return False


    # 克隆项目
    def clone(self, addr, path=None, name=None):
        import os
        repo = addr.split("/").pop().split(".git", 1)[0]
        if not self.exists(addr, path, name):
            echo(f"开始下载 {repo}")
            if path is None and name is None:
                path = os.getcwd()
                name = repo
            elif path is not None and name is None:
                name = repo
            elif path is None and name is not None:
                path = os.getcwd()
            !git clone {addr} "{path}/{name}" --recurse-submodules
        else:
            echo(f"{repo} 已存在")


    # 拉取更新
    def pull(self, path):
        import os
        if self.exists(path=path) and self.exists(path + "/.git"):
            repo = os.path.normpath(path).split("/" and "\\").pop()
            echo(f"更新 {repo} 中")
            if self.is_git_pointer_offset(path=path):
                self.fix_pointer(path=path)
            !git -C "{path}" pull --recurse-submodules
        else:
            echo(f"路径不存在")


    # 切换到指定版本
    def checkout(self, path, commit=None):
        import os
        if self.exists(path=path) and self.exists(path + "/.git"):
            repo = os.path.normpath(path).split("/" and "\\").pop()
            if commit is not None:
                echo(f"切换 {repo} 版本中")
                !git -C "{path}" reset --hard {commit} --recurse-submodules
            else:
                echo(f"未指定 {repo} 版本")
        else:
            echo("路径不存在")


    # 修复分支漂移
    def fix_pointer(self, path):
        import os
        import subprocess
        if self.exists(path=path):
            repo = os.path.normpath(path).split("/" and "\\").pop()
            echo(f"修复 {repo} 分支游离中")
            !git -C "{path}" remote prune origin
            !git -C "{path}" submodule init
            result = subprocess.run(f"git -C \"{path}\" branch -a | grep /HEAD", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout
            print(result)
            branch = result.split("/").pop()
            !git -C "{path}" checkout {branch} # 切换到主分支
            !git -C "{path}" reset --recurse-submodules --hard origin/{branch}
            !git -C "{path}" reset --recurse-submodules --hard HEAD
            !git -C "{path}" restore --recurse-submodules --source=HEAD :/
        else:
            echo("路径不存在")


    # 检测分支是否漂移
    def is_git_pointer_offset(self, path):
        if self.exists(path=path):
            import subprocess
            result = subprocess.run(f"git -C \"{path}\" symbolic-ref HEAD", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).returncode
            if result == 0:
                return True
            else:
                echo("检测到出现分支游离")
                return False


    # 输出版本信息
    def git_show_ver(self, path):
        import subprocess
        if self.exists(path=path) and self.exists(path + "/.git"):
            result = subprocess.run(f"git -C \"{path}\" symbolic-ref HEAD", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).returncode
            if result == 0:
                cmd1 = f"git -C \"{path}\" rev-parse --short HEAD"
                cmd2 = f"git -C \"{path}\" show -s --format=\"%cd\" --date=format:\"%Y-%m-%d %H:%M:%S\""
                commit = subprocess.run(f"git -C \"{path}\" symbolic-ref HEAD", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout.replace("refs/heads/", "").replace("\n","")
                commit = commit + " "  + subprocess.run(cmd1, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout.replace("\n","")
                commit = commit + " "  + subprocess.run(cmd2, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout.replace("\n","")
            else:
                cmd1 = f"git -C \"{path}\" rev-parse --short HEAD"
                cmd2 = f"git -C \"{path}\" show -s --format=\"%cd\" --date=format:\"%Y-%m-%d %H:%M:%S\""
                commit = subprocess.run(cmd1, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout.replace("\n","")
                commit = commit + " "  + subprocess.run(cmd2, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout.replace("\n","")
                commit = commit + " (分支游离)"
            return path.split("/").pop() + ": " + commit
        else:
            return "(非 Git 安装)"


    def log(self, path):
        import subprocess
        if self.exists(path=path) and self.exists(path + "/.git"):
            result = subprocess.run(f"git -C \"{path}\" symbolic-ref HEAD", stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).returncode
            if result == 0:
                cmd = f"git -C \"{path}\" log --all --date=short --pretty=format:\"%h %cd\" --date=format:\"%Y-%m-%d | %H:%M:%S\""
            return path.split("/").pop() + " 版本信息: \n" + subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, shell=True).stdout
        else:
            return path.split("/").pop() + " 非 Git 安装"


    def test_github_mirror(self, mirror):
        import os
        import subprocess
        path = self.WORKSPACE + "/empty"
        for i in mirror:
            test_repo = i.replace("term_sd_git_user/term_sd_git_repo","licyk/empty")
            avaliable_mirror = i.replace("/term_sd_git_user/term_sd_git_repo","")
            mirror_url = i.replace("term_sd_git_user/term_sd_git_repo","")
            echo(f"测试 Github 镜像源: {mirror_url}")
            if os.path.exists(path):
                !rm -rf "{path}"
            result = subprocess.run(f"git clone {test_repo} {path}")
            if os.path.exists(path):
                !rm -rf "{path}"
            if result.returncode == 0:
                return avaliable_mirror
        return False


    def set_github_mirror(self, mirror):
        import os
        avaliable_mirror = self.test_github_mirror(mirror)
        git_config_path = self.WORKSPACE + "/.gitconfig"
        if avaliable_mirror is not False:
            echo(f"设置 GIthub 镜像源: {avaliable_mirror}")
            if os.path.exists(git_config_path):
                !rm -rf "{git_config_path}"
            os.environ["GIT_CONFIG_GLOBAL"] = git_config_path
            !git config --global url.{avaliable_mirror}.insteadOf https://github.com
        else:
            echo("未找到可用 Github 镜像源, 取消设置 GIthub 镜像源")


    def unset_github_mirror(self):
        import os
        if "GIT_CONFIG_GLOBAL" in os.environ:
            echo("删除 Github 镜像源配置")
            os.unsetenv("GIT_CONFIG_GLOBAL")
        else:
            echo("Github 镜像源未设置, 无需删除")



# TUNNEL
class TUNNEL:
    LOCALHOST_RUN = "localhost.run"
    REMOTE_MOE = "remote.moe"
    WORKSPACE = ""
    WORKFOLDER = ""
    PORT = ""


    def __init__(self, workspace, workfolder, port) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder
        self.PORT = port


    # ngrok内网穿透
    def ngrok(self, ngrok_token: str):
        from pyngrok import conf, ngrok
        conf.get_default().auth_token = ngrok_token
        conf.get_default().monitor_thread = False
        port = self.PORT
        ssh_tunnels = ngrok.get_tunnels(conf.get_default())
        if len(ssh_tunnels) == 0:
            ssh_tunnel = ngrok.connect(port, bind_tls=True)
            return ssh_tunnel.public_url
        else:
            return ssh_tunnels[0].public_url


    # cloudflare内网穿透
    def cloudflare(self):
        from pycloudflared import try_cloudflare
        port = self.PORT
        urls = try_cloudflare(port).tunnel
        return urls


    from typing import Union
    from pathlib import Path

    # 生成ssh密钥
    def gen_key(self, path: Union[str, Path]) -> None:
        import subprocess
        import shlex
        from pathlib import Path
        path = Path(path)
        arg_string = f'ssh-keygen -t rsa -b 4096 -N "" -q -f {path.as_posix()}'
        args = shlex.split(arg_string)
        subprocess.run(args, check=True)
        path.chmod(0o600)


    # ssh内网穿透
    def ssh_tunnel(self, host: str) -> None:
        import subprocess
        import atexit
        import shlex
        import re
        import os
        from pathlib import Path
        from tempfile import TemporaryDirectory

        ssh_name = "id_rsa"
        ssh_path = Path(self.WORKSPACE) / ssh_name
        port = self.PORT

        tmp = None
        if not ssh_path.exists():
            try:
                self.gen_key(ssh_path)
            # write permission error or etc
            except subprocess.CalledProcessError:
                tmp = TemporaryDirectory()
                ssh_path = Path(tmp.name) / ssh_name
                self.gen_key(ssh_path)

        arg_string = f"ssh -R 80:127.0.0.1:{port} -o StrictHostKeyChecking=no -i {ssh_path.as_posix()} {host}"
        args = shlex.split(arg_string)

        tunnel = subprocess.Popen(
            args, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)
        if tmp is not None:
            atexit.register(tmp.cleanup)

        tunnel_url = ""
        LOCALHOST_RUN = self.LOCALHOST_RUN
        lines = 27 if host == LOCALHOST_RUN else 5
        localhostrun_pattern = re.compile(r"(?P<url>https?://\S+\.lhr\.life)")
        remotemoe_pattern = re.compile(r"(?P<url>https?://\S+\.remote\.moe)")
        pattern = localhostrun_pattern if host == LOCALHOST_RUN else remotemoe_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")

            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                if lines == 27:
                    os.environ['LOCALHOST_RUN'] = tunnel_url
                    return tunnel_url
                else:
                    os.environ['REMOTE_MOE'] = tunnel_url
                    return tunnel_url
                # break
        else:
            echo(f"启动 {host} 内网穿透失败")


    # localhost.run穿透
    def localhost_run(self):
        urls = self.ssh_tunnel(self.LOCALHOST_RUN)
        return urls


    # remote.moe内网穿透
    def remote_moe(self):
        urls = self.ssh_tunnel(self.REMOTE_MOE)
        return urls


    # gradio内网穿透
    def gradio(self):
        import subprocess
        import shlex
        import atexit
        import re
        port = self.PORT
        cmd = f"gradio-tunneling --port {port}"
        cmd = shlex.split(cmd)
        tunnel = subprocess.Popen(
            cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding="utf-8"
        )

        atexit.register(tunnel.terminate)

        tunnel_url = ""
        lines = 5
        gradio_pattern = re.compile(r"(?P<url>https?://\S+\.gradio\.live)")
        pattern = gradio_pattern

        for _ in range(lines):
            line = tunnel.stdout.readline()
            if line.startswith("Warning"):
                print(line, end="")
            url_match = pattern.search(line)
            if url_match:
                tunnel_url = url_match.group("url")
                return tunnel_url
        else:
            echo(f"启动 Gradio 内网穿透失败")


    # 启动内网穿透
    def start(self, ngrok=False, ngrok_token=None, cloudflare=False, remote_moe=False, localhost_run=False, gradio=False):
        if cloudflare is True or ngrok is True or ngrok_token is not None or remote_moe is True or localhost_run is True or gradio is True:
            echo("启动内网穿透")

        if cloudflare is True:
            cloudflare_url = self.cloudflare()
        else:
            cloudflare_url = None

        if ngrok is True and ngrok_token is not None:
            ngrok_url = self.ngrok(ngrok_token)
        else:
            ngrok_url = None

        if remote_moe is True:
            remote_moe_url = self.remote_moe()
        else:
            remote_moe_url = None

        if localhost_run is True:
            localhost_run_url = self.localhost_run()
        else:
            localhost_run_url = None

        if gradio is True:
            gradio_url = self.gradio()
        else:
            gradio_url = None

        echo("下方为访问地址")
        print("==================================================================================")
        echo(f"CloudFlare: {cloudflare_url}")
        echo(f"Ngrok: {ngrok_url}")
        echo(f"remote.moe: {remote_moe_url}")
        echo(f"localhost_run: {localhost_run_url}")
        echo(f"Gradio: {gradio_url}")
        print("==================================================================================")



# ENV
class ENV:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 准备ipynb笔记自身功能的依赖
    def prepare_env_depend(self, use_mirror=True):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        echo("安装自身组件依赖")
        !pip install pyngrok pycloudflared gradio-tunneling {pip_mirror}
        !apt install aria2 ssh google-perftools -y


    # 安装pytorch和xformers
    def prepare_torch(self, torch_ver, xformers_ver, use_mirror=False):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        if torch_ver != "":
            echo("安装 PyTorch")
            !pip install {torch_ver} {pip_mirror}
        if xformers_ver != "":
            echo("安装 xFormers")
            !pip install {xformers_ver} {pip_mirror}


    # 安装requirements.txt依赖
    def install_requirements(self, path, use_mirror=False):
        import os
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"
        if os.path.exists(path):
            echo("安装依赖")
            !pip install -r "{path}" {pip_mirror}
        else:
            echo("依赖文件路径为空")


    # python软件包安装
    # 可使用的操作:
    # 安装: install -> install
    # 仅安装: install_single -> install --no-deps
    # 强制重装: force_install -> install --force-reinstall
    # 仅强制重装: force_install_single -> install --force-reinstall --no-deps
    # 更新: update -> install --upgrade
    # 卸载: uninstall -y
    def py_pkg_manager(self, pkg, type=None, use_mirror=False):
        if use_mirror is True:
            pip_mirror = "--index-url https://mirrors.cloud.tencent.com/pypi/simple --find-links https://mirror.sjtu.edu.cn/pytorch-wheels/cu121/torch_stable.html"
        else:
            pip_mirror = "--index-url https://pypi.python.org/simple --find-links https://download.pytorch.org/whl/cu121/torch_stable.html"

        if type == "install":
            func = "install"
            args = ""
        elif type == "install_single":
            func = "install"
            args = "--no-deps"
        elif type == "force_install":
            func = "install"
            args = "--force-reinstall"
        elif type == "force_install_single":
            func = "install"
            args = "install --force-reinstall --no-deps"
        elif type == "update":
            func = "install"
            args = "--upgrade"
        elif type == "uninstall":
            func = "uninstall"
            args = "-y"
            pip_mirror = ""
        else:
            echo(f"未知操作: {type}")
            return
        echo(f"执行操作 :pip {func} {pkg} {args} {pip_mirror}")
        !pip {func} {pkg} {args} {pip_mirror}


    # 配置内存优化
    def tcmalloc(self):
        echo("配置内存优化")
        import os
        os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4"


    # 适用于colab的内存优化
    def tcmalloc_colab(self):
        echo("配置内存优化")
        import os
        aria2 = ARIA2(self.WORKSPACE, self.WORKFOLDER)
        path = self.WORKSPACE
        libtcmalloc_path = self.WORKSPACE + "/libtcmalloc_minimal.so.4"
        aria2.aria2("https://github.com/licyk/term-sd/releases/download/archive/libtcmalloc_minimal.so.4", path, "libtcmalloc_minimal.so.4")
        os.environ["LD_PRELOAD"] = libtcmalloc_path



# MANAGER
class MANAGER:
    WORKSPACE = ""
    WORKFOLDER = ""


    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 清理ipynb笔记的输出
    def clear_up(self, opt):
        from IPython.display import clear_output
        clear_output(wait=opt)


    # 检查gpu是否可用
    def check_gpu(self):
        echo("检测 GPU 是否可用")
        import tensorflow as tf
        echo(f"TensorFlow 版本: {tf.__version__}")
        if tf.test.gpu_device_name():
            echo("GPU 可用")
        else:
            echo("GPU 不可用")
            raise Exception("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")


    # 配置google drive
    def config_google_drive(self):
        echo("挂载 Google Drive")
        import os
        if not os.path.exists('/content/drive/MyDrive'):
            from google.colab import drive
            drive.mount('/content/drive')
            echo("Google Dirve 挂载完成")
        else:
            echo("Google Drive 已挂载")

        # 检测并创建输出文件夹
        if os.path.exists('/content/drive/MyDrive'):
            if not os.path.exists('/content/drive/MyDrive/fooocus_output'):
                echo("在 Google Drive 创建 fooocus_ouput 文件夹")
                !mkdir -p /content/drive/MyDrive/fooocus_output
        else:
            raise Exception("未挂载 Google Drive，请重新挂载后重试！")


    def set_huggingface_mirror(self ,mirror):
        import os
        echo(f"设置 HuggingFace 镜像源: {mirror}")
        os.environ["HF_ENDPOINT"] = mirror


    def unset_huggingface_mirror(self):
        import os
        if "HF_ENDPOINT" in os.environ:
            echo("删除 HuggingFace 镜像源配置")
            os.unsetenv("HF_ENDPOINT")
        else:
            echo("HuggingFace 镜像源未设置, 无需删除")


    def hf_link_to_mirror_link(self, hf_mirror, url):
        if isinstance(url, str):
            return url.replace("https://huggingface.co",hf_mirror)
        else:
            mirror_url = url
            j = 0
            for i in mirror_url:
                tmp = i.replace("https://huggingface.co",hf_mirror)
                mirror_url[j] = tmp
                j += 1
            return mirror_url


    def select_list(self, data, name):
        # https://stackoverflow.com/questions/57219796/ipywidgets-dynamic-creation-of-checkboxes-and-selection-of-data
        # https://gist.github.com/MattJBritton/9dc26109acb4dfe17820cf72d82f1e6f
        import ipywidgets as widgets
        names = [] # 可选择的列表
        checkbox_objects = [] # 按钮对象
        for key in data:
            value = key[1]
            key = key[0].split("/").pop()
            if value == 1:
                select = True
            else:
                select = False
            checkbox_objects.append(widgets.Checkbox(value=select, description=key, )) # 扩展按钮列表
            names.append(key)

        arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

        ui = widgets.VBox(children=checkbox_objects) # 创建widget

        selected_data = []
        select_value = [] # 存储每个模型选择情况
        url_list = [] # 地址列表
        def select_data(**kwargs): # 每次点击按钮时都会执行
            selected_data.clear()
            select_value.clear()
            for key in kwargs:
                if kwargs[key] is True:
                    selected_data.append(key)
                    select_value.append(True)
                else:
                    select_value.append(False)

            list = ""
            for i in selected_data: # 已选择的模型列表(模型名称)
                list = f"{list}\n- {i}"
            print(f"已选择列表: {list}")
            j = 0
            url_list.clear()
            for i in select_value: # 返回的地址列表
                if i is True:
                    url_list.append(data[j][0])
                j += 1

        out = widgets.interactive_output(select_data, arg_dict)
        ui.children = [*ui.children, out]
        ui = widgets.Accordion(children=[ui,], titles=(name,))
        #display(ui, out)
        display(ui)
        return url_list


    def select_button(self, name, normal_value):
        import ipywidgets as widgets
        return widgets.Checkbox(
            value=normal_value,
            description=name,
            disabled=False,
            indent=False
        )


    def text_input(self, name, notice, normal_text=""):
        import ipywidgets as widgets
        return widgets.Textarea(
            value=normal_text,
            placeholder=notice ,
            description=name,
            disabled=False
        )


    def dropdown(self, name, list, normal_value):
        import ipywidgets as widgets
        widgets.Dropdown(
            options=list,
            value=normal_value,
            description=name,
            disabled=False,
        )



# FOOOCUS
class FOOOCUS(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 7865)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    # 下载大模型
    def get_sd_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/checkpoints"
        super().aria2(url, path, filename)


    def get_sd_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_sd_model(i, i.split("/").pop())


    # 下载lora模型
    def get_lora_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/loras"
        super().aria2(url, path, filename)


    def get_lora_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_lora_model(i, i.split("/").pop())


    # 下载配置文件
    def install_config(self):
        path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("下载配置文件")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_config.json", path + "/presets", "custom.json")
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_zh_cn.json", path + "/language", "zh.json")


    def install_config_colab(self):
        import os
        path = self.WORKSPACE + "/" + self.WORKFOLDER
        path_config = path + "/config.txt"
        echo("下载路径配置文件")
        if os.path.exists(path_config):
            !rm -rf "{path_config}"
        self.aria2("https://github.com/licyk/term-sd/releases/download/archive/fooocus_path_config_colab.json", path, "config.txt")


    # 安装fooocus
    def install(self, torch_ver, xformers_ver, sd, lora, use_mirror):
        import os
        self.check_gpu()
        self.prepare_env_depend(use_mirror)
        self.clone("https://github.com/lllyasviel/Fooocus", self.WORKSPACE)
        os.chdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        self.prepare_torch(torch_ver, xformers_ver, use_mirror)
        req_file = self.WORKSPACE + "/" + self.WORKFOLDER + "/requirements_versions.txt"
        self.install_requirements(req_file, use_mirror)
        self.install_config()
        self.tcmalloc()
        echo("下载模型")
        self.get_sd_model_from_list(sd)
        self.get_lora_model_from_list(lora)


    def update(self):
        import os
        fooocus_path = self.WORKSPACE + "/" + self.WORKFOLDER
        self.pull(fooocus_path)



# INVOKEAI
class INVOKEAI(ARIA2, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 9090)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def get_sd_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/checkpoint"
        super().aria2(url, path, filename)


    def get_sd_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_sd_model(i, i.split("/").pop())


    def get_lora_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/loras"
        super().aria2(url, path, filename)


    def get_lora_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_lora_model(i, i.split("/").pop())


    def get_embedding_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/embeddings"
        super().aria2(url, path, filename)


    def get_embedding_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_embedding_model(i, i.split("/").pop())

    def get_vae_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/models/vae"
        super().aria2(url, path, filename)


    def get_vae_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_vae_model(i, i.split("/").pop())


    def install(self, torch_ver, xformers_ver, sd, lora, vae, embedding, use_mirror):
        import os
        self.check_gpu()
        self.prepare_env_depend(use_mirror)
        os.mkdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        os.chdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        self.prepare_torch("", xformers_ver)
        self.py_pkg_manager("invokeai", "install", use_mirror)
        self.tcmalloc()
        echo("下载模型")
        self.get_sd_model_from_list(sd)
        self.get_lora_model_from_list(lora)
        self.get_vae_model_from_list(vae)
        self.get_embedding_model_from_list(embedding)


    def update(self):
        echo("更新 InvokeAI")
        !pip install invokeai -U



# SD_TRAINER
class SD_TRAINER(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 28000)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def get_sd_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/sd-models"
        super().aria2(url, path, filename)


    def get_sd_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_sd_model(i, i.split("/").pop())


    def install(self, torch_ver, xformers_ver, sd, use_mirror):
        import os
        self.check_gpu()
        self.prepare_env_depend(use_mirror)
        self.clone("https://github.com/Akegarasu/lora-scripts", self.WORKSPACE)
        os.chdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        self.prepare_torch(torch_ver, xformers_ver)
        req_file = self.WORKSPACE + "/" + self.WORKFOLDER + "/requirements.txt"
        self.install_requirements(req_file, use_mirror)
        self.tcmalloc()
        self.get_sd_model_from_list(sd)


    def update(self):
        import os
        sd_trainer_path = self.WORKSPACE + "/" + self.WORKFOLDER
        self.pull(sd_trainer_path)


    def get_core_ver(self):
        sd_trainer_path =  self.WORKSPACE + "/" + self.WORKFOLDER
        print(self.log(sd_trainer_path))


    def set_core_ver(self, commit):
        sd_trainer_path =  self.WORKSPACE + "/" + self.WORKFOLDER
        self.checkout(sd_trainer_path, commit)


    def show_ver(self):
        import os
        sd_trainer_path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("SD Trainer 版本:")
        print(self.git_show_ver(sd_trainer_path))


# KOHYA GUI
class KOHYA_GUI(ARIA2, GIT, TUNNEL, MANAGER, ENV):
    WORKSPACE = ""
    WORKFOLDER = ""

    tun = TUNNEL(WORKSPACE, WORKFOLDER, 7860)

    def __init__(self, workspace, workfolder) -> None:
        self.WORKSPACE = workspace
        self.WORKFOLDER = workfolder


    def get_sd_model(self, url, filename):
        path = self.WORKSPACE + "/" + self.WORKFOLDER + "/sd-models"
        super().aria2(url, path, filename)

    def get_sd_model_from_list(self, list):
        for i in list:
            if i != "":
                self.get_sd_model(i, i.split("/").pop())


    def install(self, torch_ver, xformers_ver, sd, use_mirror):
        import os
        self.check_gpu()
        self.prepare_env_depend(use_mirror)
        self.clone("https://github.com/bmaltais/kohya_ss", self.WORKSPACE)
        os.chdir(f"{self.WORKSPACE}/{self.WORKFOLDER}")
        self.prepare_torch(torch_ver, xformers_ver)
        req_file = self.WORKSPACE + "/" + self.WORKFOLDER + "/requirements.txt"
        self.install_requirements(req_file, use_mirror)
        self.tcmalloc()
        self.get_sd_model_from_list(sd)


    def update(self):
        import os
        kohya_gui_path = self.WORKSPACE + "/" + self.WORKFOLDER
        self.pull(kohya_gui_path)


    def get_core_ver(self):
        kohya_gui_path =  self.WORKSPACE + "/" + self.WORKFOLDER
        print(self.log(kohya_gui_path))


    def set_core_ver(self, commit):
        kohya_gui_path =  self.WORKSPACE + "/" + self.WORKFOLDER
        self.checkout(kohya_gui_path, commit)


    def show_ver(self):
        import os
        kohya_gui_path = self.WORKSPACE + "/" + self.WORKFOLDER
        echo("Kohya_GUI 版本:")
        print(self.git_show_ver(kohya_gui_path))


echo("初始化功能完成")

:: 初始化功能完成


In [ ]:
#@title 👇 参数配置
try:
    i = INIT_CONFIG
    echo("尝试安装 ipywidgets 组件")
    !pip install ipywidgets -qq
    from IPython.display import clear_output
    clear_output(wait=False)
    INIT_CONFIG_1 = 1
except:
    raise Exception("未初始化功能")

import ipywidgets as widgets

WEBUI = ""
WORKSPACE = ""
WORKFOLDER = ""
USE_MIRROR = False
USE_NGROK = True
NGROK_TOKEN = ""
USE_CLOUDFLARE = True
USE_REMOTE_MOE = True
USE_LOCALHOST_RUN = True
USE_GRADIO_SHARE = False
TORCH_VER = ""
XFORMERS_VER = ""
GITHUB_MIRROR = [
    "https://ghproxy.net/https://github.com/term_sd_git_user/term_sd_git_repo",
    "https://mirror.ghproxy.com/https://github.com/term_sd_git_user/term_sd_git_repo",
    "https://gh-proxy.com/https://github.com/term_sd_git_user/term_sd_git_repo",
    "https://ghps.cc/https://github.com/term_sd_git_user/term_sd_git_repo",
    "https://gh.idayer.com/https://github.com/term_sd_git_user/term_sd_git_repo",
    "https://gitclone.com/github.com/term_sd_git_user/term_sd_git_repo"
]
HUGGINGFACE_MIRROR = "https://hf-mirror.com"
sd_model = [
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/v1-5-pruned-emaonly.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/animefull-final-pruned.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/Counterfeit-V3.0_fp16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/cetusMix_Whalefall2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/cuteyukimixAdorable_neochapter3.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/ekmix-pastel-fp16-no-ema.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/ex2K_sse2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/kohakuV5_rev2.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/meinamix_meinaV11.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/oukaStar_10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/pastelMixStylizedAnime_pastelMixPrunedFP16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/rabbit_v6.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/sweetSugarSyndrome_rev15.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/AnythingV5Ink_ink.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/bartstyledbBlueArchiveArtStyleFineTunedModel_v10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/meinapastel_v6Pastel.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/qteamixQ_omegaFp16.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sd_1.5/tmndMix_tmndMixSPRAINBOW.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/sd_xl_base_1.0_0.9vae.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.0.safetensors", 1],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/AnythingXL_xl.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/abyssorangeXLElse_v10.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animaPencilXL_v200.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/animagine-xl-3.1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/heartOfAppleXL_v20.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/baxlBartstylexlBlueArchiveFlatCelluloid_xlv1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohaku-xl-delta-rev1.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/kohakuXLEpsilon_rev1.safetensors", 1],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/nekorayxl_v06W3.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/CounterfeitXL-V1.0.safetensors", 0],
    ["https://huggingface.co/licyk/sd-model/resolve/main/sdxl_1.0/ponyDiffusionV6XL_v6StartWithThisOne.safetensors", 0]
]
lora = [
    ["https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/AnimagineXLV3_Style_Difference_AnimeFace.safetensors", 1],
    ["https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/AnimagineXLV3_Style_Difference_Saturation.safetensors", 0],
    ["https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/AnimagineXLV3_Style_Difference_EdgeEmphasys.safetensors", 0],
    ["https://huggingface.co/licyk/sd-lora/resolve/main/sdxl/style/AnimagineXLV3_Style_Difference_bodyshadow.safetensors", 0],
    ["https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_offset_example-lora_1.0.safetensors", 0]
]
embedding = [
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/EasyNegative.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/EasyNegativeV2.safetensors", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/bad-artist-anime.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/bad-artist.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/bad-hands-5.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/bad-image-v2-39000.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/bad_prompt_version2.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/ng_deepnegative_v1_75t.pt", 1],
    ["https://huggingface.co/licyk/sd-embeddings/resolve/main/sd_1.5/verybadimagenegative_v1.3.pt", 1]
]
vae = [
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-ema-560000-ema-pruned.safetensors", 1],
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sd_1.5/vae-ft-mse-840000-ema-pruned.safetensors", 1],
    ["https://huggingface.co/licyk/sd-vae/resolve/main/sdxl_1.0/sdxl_fp16_fix_vae.safetensors", 1]
]
vae_approx = [
    ["https://huggingface.co/licyk/sd-vae/resolve/main/vae-approx/model.pt", 1],
    ["https://huggingface.co/licyk/sd-vae/resolve/main/vae-approx/vaeapprox-sdxl.pt", 1]
]
upscale = [
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/ESRGAN/4x-UltraSharp.pth", 1],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/ESRGAN/ESRGAN_4x.pth", 1],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/ESRGAN/4x_NMKD-Superscale-SP_178000_G.pth", 1],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/RealESRGAN/RealESRGAN_x4plus.pth", 0],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/RealESRGAN/RealESRGAN_x4plus_anime_6B.pth", 1],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/DAT/DAT_x2.pth", 0],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/DAT/DAT_x3.pth", 1],
    ["https://huggingface.co/licyk/sd-upscaler-models/resolve/main/DAT/DAT_x4.pth", 1]
]

WEBUI_LIST = ["sd_webui", "comfyui", "fooocus", "invokeai", "sd_trainer", "kohya_gui"]

manager = MANAGER(WORKSPACE, WORKFOLDER)
gi = GIT(WORKSPACE, WORKFOLDER)

workspace_state = manager.text_input("工作区路径: ", "请输入工作区路径")
webui_state = widgets.Dropdown(options=WEBUI_LIST, value="sd_webui", description="使用的 WebUI: ", disabled=False)
torch_ver_state = manager.text_input("PyTorch 版本: ", "请输入 PyTorch 版本", "torch==2.2.1+cu121 torchvision==0.17.1+cu121")
xformers_ver_state = manager.text_input("xFormers 版本: ", "请输入 xFormers 版本", "xformers==0.0.25")
use_mirror_state = manager.select_button("使用镜像源", False)
is_colab_state = manager.select_button("使用 Colab", False)
use_ngrok_state = manager.select_button("使用 Ngrok 内网穿透（需填写 Ngrok Token）", False)
ngrok_token_state = manager.text_input("Ngrok Token: ", "请输入 Ngrok Token")
use_cloudflare_state = manager.select_button("使用 CloudFlare 内网穿透", True)
use_remote_moe_state = manager.select_button("使用 remote.moe 内网穿透", True)
use_localhost_run_state = manager.select_button("使用 localhost.run 内网穿透", True)
use_gradio_share_state = manager.select_button("使用 Gradio 内网穿透", False)
display(workspace_state, webui_state, torch_ver_state, xformers_ver_state, use_mirror_state, use_ngrok_state, ngrok_token_state, use_cloudflare_state, use_remote_moe_state, use_localhost_run_state, use_gradio_share_state)


sd_model_list = manager.select_list(sd_model,"Stable Diffusion 模型")
lora_list = manager.select_list(lora, "LoRA 模型")
embedding_list = manager.select_list(embedding, "Embedding 模型")
vae_list = manager.select_list(vae, "VAE 模型")
vae_approx_list = manager.select_list(vae_approx, "VAE-approx 模型")
upscale_list = manager.select_list(upscale, "放大模型")

:: 尝试安装 ipywidgets 组件


In [ ]:
#@title 👇 应用参数配置
try:
    i = INIT_CONFIG_1
    INIT_CONFIG_1 = 1
    INIT_CONFIG_2 = 1
except:
    raise Exception("未运行\"参数配置\"单元")

import os
os.chdir(WORKSPACE)
WORKSPACE = workspace_state.value.rstrip("/").rstrip("\\")
WEBUI = webui_state.value
TORCH_VER = torch_ver_state.value
XFORMERS_VER = xformers_ver_state.value
USE_MIRROR = use_mirror_state.value
IS_COLAB = is_colab_state.value
USE_NGROK = use_ngrok_state.value
NGROK_TOKEN = ngrok_token_state.value
USE_CLOUDFLARE = use_cloudflare_state.value
USE_REMOTE_MOE = use_remote_moe_state.value
USE_LOCALHOST_RUN = use_localhost_run_state.value
USE_GRADIO_SHARE = use_gradio_share_state.value

if WORKSPACE != "":
    if WEBUI == "fooocus":
        WORKFOLDER = "Fooocus"
        fooocus = FOOOCUS(WORKSPACE, WORKFOLDER)
    elif WEBUI == "invokeai":
        WORKFOLDER = "InvokeAI"
        invokeai = INVOKEAI(WORKSPACE, WORKFOLDER)
    elif WEBUI == "sd_trainer":
        WORKFOLDER = "lora-scripts"
        sd_trainer = SD_TRAINER(WORKSPACE, WORKFOLDER)
    elif WEBUI == "kohya_gui":
        WORKFOLDER = "kohya_ss"
        kohya_gui = KOHYA_GUI(WORKSPACE, WORKFOLDER)
else:
    raise Exception("未填写工作区路径")

manager = MANAGER(WORKSPACE, WORKFOLDER)

if USE_MIRROR:
    sd_model = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, sd_model_list)
    lora = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, lora_list)
    embedding = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, embedding_list)
    vae = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, vae_list)
    vae_approx = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, vae_approx_list)
    upscale = manager.hf_link_to_mirror_link(HUGGINGFACE_MIRROR, upscale_list)
    manager.set_huggingface_mirror(HUGGINGFACE_MIRROR)
    gi.set_github_mirror(GITHUB_MIRROR)
echo("参数设置完成")

In [ ]:
#@title 👇 安装
try:
    i = INIT_CONFIG_2
    INIT_CONFIG_2 = 1
    INIT_CONFIG_3 = 1
except:
    raise Exception("未运行\"参数设置\"单元")

import os
os.chdir(WORKSPACE)
echo(f"开始安装 {WEBUI}")
if WEBUI == "fooocus":
    fooocus.install(TORCH_VER, XFORMERS_VER, sd_model_list, lora_list, USE_MIRROR)
    if IS_COLAB is True:
        fooocus.tcmalloc_colab()
        fooocus.install_config_colab()
elif WEBUI == "invokeai":
    invokeai.install(TORCH_VER, XFORMERS_VER, sd_model_list, lora_list, vae_list, embedding_list, USE_MIRROR)
    if IS_COLAB is True:
        fooocus.tcmalloc_colab()
elif WEBUI == "sd_trainer":
    sd_trainer.install(TORCH_VER, XFORMERS_VER, sd_model_list, USE_MIRROR)
    if IS_COLAB is True:
        sd_trainer.tcmalloc_colab()
elif WEBUI == "kohya_gui":
    kohya_gui.install(TORCH_VER, XFORMERS_VER, sd_model_list, USE_MIRROR)
    if IS_COLAB is True:
        kohya_gui.tcmalloc_colab()
manager.clear_up(False)
echo(f"{WEBUI} 安装完成")

In [ ]:
#@title 👇 启动
try:
    i = INIT_CONFIG_3
    INIT_CONFIG_3 = 1
except:
    raise Exception("未运行\"安装\"单元")

import os
os.chdir(WORKSPACE + "/" + WORKFOLDER)
echo(f"启动 {WEBUI}")
if WEBUI == "fooocus":
    fooocus.tun.start(ngrok=USE_NGROK, ngrok_token=NGROK_TOKEN, cloudflare=USE_CLOUDFLARE, remote_moe=USE_REMOTE_MOE, localhost_run=USE_LOCALHOST_RUN, gradio=USE_GRADIO_SHARE)
    !python "{WORKSPACE}"/Fooocus/launch.py --preset custom --language zh
elif WEBUI == "invokeai":
    invokeai.tun.start(ngrok=USE_NGROK, ngrok_token=NGROK_TOKEN, cloudflare=USE_CLOUDFLARE, remote_moe=USE_REMOTE_MOE, localhost_run=USE_LOCALHOST_RUN, gradio=USE_GRADIO_SHARE)
    !invokeai-web --root "{WORKSPACE}"/InvokeAI/invokeai
elif WEBUI == "sd_trainer":
    sd_trainer.tun.start(ngrok=USE_NGROK, ngrok_token=NGROK_TOKEN, cloudflare=USE_CLOUDFLARE, remote_moe=USE_REMOTE_MOE, localhost_run=USE_LOCALHOST_RUN, gradio=USE_GRADIO_SHARE)
    !python "{WORKSPACE}"/lora-scripts/gui.py
elif WEBUI == "kohya_gui":
    kohya_gui.tun.start(ngrok=USE_NGROK, ngrok_token=NGROK_TOKEN, cloudflare=USE_CLOUDFLARE, remote_moe=USE_REMOTE_MOE, localhost_run=USE_LOCALHOST_RUN, gradio=USE_GRADIO_SHARE)
    !python "{WORKSPACE}"/kohya_ss/kohya_gui.py --language zh-CN
manager.clear_up(False)
echo(f"{WEBUI} 已关闭")

##✨ 其他功能

In [ ]:
#@title 👇 自定义模型 / 扩展下载配置
try:
    i = INIT_CONFIG_3
    INIT_CONFIG_3 = 1
    INIT_CONFIG_4 = 1
except:
    raise Exception("未运行\"安装\"单元")

import ipywidgets as widgets

model_ = widgets.Textarea(
    value="",
    placeholder='请填写模型 / 扩展下载链接',
    description='模型 / 扩展下载链接: ',
    disabled=False
)

model_name_ = widgets.Textarea(
    value="",
    placeholder='请填写模型名称',
    description='模型名称: ',
    disabled=False
)

model_type_ = widgets.Dropdown(
    options=[("Stable Diffusion 模型（大模型）", "sd"),
             ("LoRA 模型", "lora"),
             ("Emebdding 模型", "embedding"),
             ("VAE 模型", "vae"),
             ("VAE-approx 模型", "vae-approx"),
             ("放大模型", "upscale"),
             ("ControlNet 模型", "controlnet"),
             ("SD WebUI 扩展", "extension"),
             ("ComfyUI 扩展", "custom_node")
             ],
    value="sd",
    description='模型 / 扩展种类',
)

display(model_, model_name_, model_type_)

In [ ]:
#@title 👇 自定义模型 / 扩展下载
try:
    i = INIT_CONFIG_4
except:
    raise Exception("未运行\"自定义模型 / 扩展下载配置\"单元")

model = model_.value
model_name = model_name_.value
model_type = model_type_.value

if model == "" and model_name == "":
    raise Exception("未填写模型 / 扩展链接或者模型名称")


if WEBUI == "invokeai":
    if model_type == "sd":
        invokeai.get_sd_model(model, model_name)
    if model_type == "lora":
        invokeai.get_lora_model(model, model_name)
    if model_type == "embedding":
        invokeai.get_embedding_model(model, model_name)
    if model_type == "vae":
        invokeai.get_vae_model(model, model_name)
elif WEBUI == "fooocus":
    if model_type == "sd":
        fooocus.get_sd_model(model, model_name)
    if model_type == "lora":
        fooocus.get_lora_model(model, model_name)
elif WEBUI == "sd_trainer":
    if model_type == "sd":
        sd_trainer.get_sd_model(model, model_name)
elif WEBUI == "kohya_gui":
    if model_type == "sd":
        kohya_gui.get_sd_model(model, model_name)
else:
    raise Exception(f"未知软件类型 {WEBUI}")

echo(f"{WEBUI} 模型文件扩展下载完成")

In [ ]:
#@title 👇 更新
try:
    i = INIT_CONFIG_3
    INIT_CONFIG_3 = 1
except:
    raise Exception("未运行\"安装\"单元")

import os
os.chdir(WORKSPACE + "/" + WORKFOLDER)
echo(f"启动 {WEBUI}")
if WEBUI == "fooocus":
    fooocus.update()
elif WEBUI == "invokeai":
    invokeai.update()
elif WEBUI == "sd_trainer":
    sd_trainer.update()
elif WEBUI == "kohya_gui":
    kohya_gui.update()
echo(f"更新 {WEBUI} 完成")